# 2022/09/06

In [ ]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.utils.prune as prune
import torch.nn.functional as F

from torch.utils.data import TensorDataset
from torch.utils.data import DataLoader
from torch.utils.data import random_split
from torchmetrics import CohenKappa

from scipy import io
import os

In [ ]:
import wandb
# wandb.init(project="test")

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

In [ ]:
sweep_config = {
    'method': 'grid'
    }

parameters_dict = {
    'hidden_size': {
        'values': [16,32,64]
        },
    'batch_size': {
        'values': [128]
        },
    'optimizer': {
        'values': ['adam']
        },
    'epochs': {
        'values': [400]
        },
    'learning_rate': {
        'values': [0.001,0.01]
      }, 
    }
sweep_config['parameters'] = parameters_dict

In [ ]:
sweep_id = wandb.sweep(sweep_config, project="BEEE_DY_csp_1003_1900")
config = wandb.config

In [ ]:
def load_mat_file(x_group1,y_group1,x_group2,y_group2):

    mat_file = io.loadmat( f'C:/Users/Peter/Desktop/CGX/data/CSP/DY_csp.mat')
    

    X1 = mat_file['Z1_1']
    X2 = mat_file['Z2_1']

    Y1 = mat_file['Y1_1']
    Y2 = mat_file['Y2_1']

    # K 특성에 대한 Class1 vs Class2 Data 가져오기
    x1 = torch.FloatTensor(X1)
    x2 = torch.FloatTensor(X2)

    # Y에 대한 Class1 vs Class2 Data 가져오기
    y1 = torch.LongTensor(Y1)
    y2 = torch.LongTensor(Y2)

    x_train = torch.cat([x1,x2],dim=0)
    y_train = torch.cat([y1,y2],dim=0)
    y_train = y_train-1 # y를 0~1의 정수로 만들어야함.

    x_group1 = torch.cat([x_group1.to(device),x_train.to(device)], dim=0).to(device)
    y_group1 = torch.cat([y_group1.to(device),y_train.to(device)], dim=0).to(device)

    X1 = mat_file['Z1_2']
    X2 = mat_file['Z2_2']

    Y1 = mat_file['Y1_2']
    Y2 = mat_file['Y2_2']

    # K 특성에 대한 Class1 vs Class2 Data 가져오기
    x1 = torch.FloatTensor(X1)
    x2 = torch.FloatTensor(X2)

    # Y에 대한 Class1 vs Class2 Data 가져오기
    y1 = torch.LongTensor(Y1)
    y2 = torch.LongTensor(Y2)

    x_train = torch.cat([x1,x2],dim=0)
    y_train = torch.cat([y1,y2],dim=0)
    y_train = y_train-1 # y를 0~1의 정수로 만들어야함.

    x_group2 = torch.cat([x_group2.to(device),x_train.to(device)], dim=0).to(device)
    y_group2 = torch.cat([y_group2.to(device),y_train.to(device)], dim=0).to(device)

    return x_group1, y_group1, x_group2, y_group2



In [ ]:
def build_dataset(batch_size,x_train, y_train, x_test, y_test):
    dataset_train = TensorDataset(x_train.to(device),   y_train.to(device)) # 각 tensor의 첫번째 dim이 일치해야한다
    dataset_test  = TensorDataset(x_test.to(device) ,   y_test.to(device) ) # 각 tensor의 첫번째 dim이 일치해야한다

    # Data Split
    dataset_size = len(dataset_train)
    train_size = int(dataset_size * 0.9)
    validation_size = dataset_size - train_size
    train_dataset, valid_dataset = random_split(dataset_train, [train_size, validation_size])

    train_dataloader    = DataLoader(train_dataset  ,batch_size=      batch_size  , shuffle=True, drop_last=True)
    valid_dataloader    = DataLoader(valid_dataset  ,batch_size= validation_size, shuffle=True, drop_last=True)

    # Data Split
    test_size = len(dataset_test)
    test_dataloader = DataLoader(dataset_test, batch_size= test_size, shuffle=True, drop_last=True)

    return train_dataloader, valid_dataloader, test_dataloader,  validation_size,test_size

In [ ]:
def build_optimizer(network, optimizer, learning_rate):
    if optimizer == "sgd":
        optimizer = optim.SGD(network.parameters(),
                              lr=learning_rate, momentum=0.9)
    elif optimizer == "adam":
        optimizer = optim.Adam(network.parameters(),
                               lr=learning_rate)
    return optimizer

In [ ]:
input_size = 4
n_class = 2

dtype = torch.float

class TextLSTM(nn.Module):
  def __init__(self,hidden_size):
    super(TextLSTM, self).__init__()

    self.lstm = nn.LSTM(input_size=input_size, hidden_size=hidden_size)
    self.fc_1 = nn.Linear(hidden_size, hidden_size)
    self.fc_2 = nn.Linear(hidden_size, n_class)
    

  def forward(self, hidden_and_cell, X):

    x = X.transpose(1,2)
    x = x.transpose(0,1)

    outputs1, (h_n1,c_n1) = self.lstm(x, hidden_and_cell)

    outputs = outputs1[-1]

    model = F.relu(self.fc_1(outputs))  # 최종 예측 최종 출력 층
    model = self.fc_2(model)

    return model.to(device)

In [ ]:
def train(config=None):
    # Initialize a new wandb run
    with wandb.init(config=config):
        # If called by wandb.agent, as below,
        # this config will be set by Sweep Controller
        config = wandb.config


        x_train = torch.tensor([]).to(device)
        y_train = torch.LongTensor([]).to(device)

        x_test = torch.tensor([]).to(device)
        y_test = torch.LongTensor([]).to(device)

        x_train, y_train, x_test, y_test = load_mat_file(x_train, y_train, x_test,  y_test)

        model = TextLSTM(hidden_size=config.hidden_size).to(device)
        criterion = nn.CrossEntropyLoss()
        optimizer = build_optimizer(model, config.optimizer, config.learning_rate)
        scheduler = optim.lr_scheduler.LambdaLR(optimizer=optimizer,
                                        lr_lambda=lambda epoch: 0.95 ** epoch,
                                        last_epoch=-1,
                                        verbose=False)

        model.train()
        
        train_dataloader, valid_dataloader, test_dataloader,  validation_size,test_size = build_dataset(config.batch_size, x_train.to(device),  y_train.to(device),x_test.to(device),  y_test.to(device))

        for epoch in range(config.epochs):
            rloss = 0
            for batch_idx, samples in enumerate(train_dataloader):

                x_train_mb, y_train_mb = samples

                hidden  = torch.zeros(1, config.batch_size, config.hidden_size, requires_grad=True).to(device)
                cell    = torch.zeros(1, config.batch_size, config.hidden_size, requires_grad=True).to(device)

                # Forward
                output = model((hidden, cell),  x_train_mb.to(device))

                # Cost
                loss = criterion(output.to(device), y_train_mb.squeeze().to(device))

                # if (epoch) % 100 == 0 and batch_idx % 2 == 0:
                #     print('Epoch {:3d}/{} Batch: {:2d} Cost: {:.6f}'.format(epoch, config.epochs, batch_idx, loss))
                
                # Backpropagate
                optimizer.zero_grad()
                loss.backward()
                optimizer.step()

                loss_b = loss.item()*config.batch_size
                rloss += loss_b
            loss_e = rloss/len(train_dataloader.dataset) # epoch loss 
            wandb.log({"loss": loss_e})
            if epoch % 100 == 0:
                print(f"Epoch: {epoch}, train loss: {round(loss_e,3)}")   
            scheduler.step()
        
        model.eval()
        with torch.no_grad():
            for batch_idx, samples in enumerate(test_dataloader):
                x_train_mb,  y_train_mb = samples

                hidden    = torch.zeros(1, test_size, config.hidden_size).to(device)
                cell      = torch.zeros(1, test_size, config.hidden_size).to(device)

                output = model((hidden, cell), x_train_mb.to(device))
                prediction = output.argmax(dim=1)
                correct = prediction.eq(y_train_mb.view_as(prediction)).sum().item()
                print(correct/test_size)
                wandb.log({"accuracy": correct/test_size})

In [ ]:
wandb.agent(sweep_id, train)

In [ ]:
wandb.finish()